In [1]:
import torch 
import torch.nn.functional as F
import matplotlib.pyplot as plt # diagra/figure
%matplotlib inline

In [8]:
words = open('names.txt', 'r').read().splitlines()
words[0:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [23]:
len(words)

32033

In [22]:
# build the vocabulary of character and mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [36]:
# build the dataset

block_size = 3 # contest length: how many character do we take to predict to the next one
X, Y = [], []

for w in words[:5]:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append
X = torch.tensor(X)
Y = torch.tensor(Y)

... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
... ---> a
..a ---> v
.av ---> a
ava ---> .
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [37]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [38]:
X, Y

(tensor([[ 0,  0,  0],
         [ 0,  0,  5],
         [ 0,  5, 13],
         [ 5, 13, 13],
         [13, 13,  1],
         [ 0,  0,  0],
         [ 0,  0, 15],
         [ 0, 15, 12],
         [15, 12,  9],
         [12,  9, 22],
         [ 9, 22,  9],
         [22,  9,  1],
         [ 0,  0,  0],
         [ 0,  0,  1],
         [ 0,  1, 22],
         [ 1, 22,  1],
         [ 0,  0,  0],
         [ 0,  0,  9],
         [ 0,  9, 19],
         [ 9, 19,  1],
         [19,  1,  2],
         [ 1,  2,  5],
         [ 2,  5, 12],
         [ 5, 12, 12],
         [12, 12,  1],
         [ 0,  0,  0],
         [ 0,  0, 19],
         [ 0, 19, 15],
         [19, 15, 16],
         [15, 16,  8],
         [16,  8,  9],
         [ 8,  9,  1]]),
 tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
          1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0]))

In [39]:
C = torch.randn(27, 2)

In [61]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [63]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [92]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # view() operation - '-1' means pytorch infer what it should be

In [93]:
h.shape

torch.Size([32, 100])

In [94]:
h

tensor([[-0.4695,  0.9991, -0.9416,  ..., -0.9943, -0.1683, -0.8604],
        [-0.9688,  0.9834, -0.9499,  ..., -0.9494,  0.2096, -0.9676],
        [-0.9991, -0.0368, -0.5022,  ...,  0.7477,  0.9514, -0.6616],
        ...,
        [ 0.9998,  0.9988,  0.0255,  ...,  0.9988, -0.7663, -0.9847],
        [-0.7399,  0.1629, -0.9391,  ...,  0.5997,  0.1699,  0.9962],
        [ 0.8522, -0.6154, -1.0000,  ...,  0.9118, -0.9218, -0.9993]])

In [95]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [96]:
logits = h @ W2 + b2

In [97]:
logits.shape

torch.Size([32, 27])

In [98]:
counts = logits.exp()

In [99]:
prob = counts / counts.sum(1, keepdims=True)

In [100]:
prob.shape

torch.Size([32, 27])

In [103]:
torch.arange(32)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [104]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [112]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(13.1199)

In [114]:
# -------------------------------------------------------

In [115]:
X.shape, Y.shape # dataset

(torch.Size([32, 3]), torch.Size([32]))

In [117]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1= torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [118]:
sum(p.nelement () for p in parameters) # number of parameters

3481

In [ ]:
# forward pass 
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 #(32, 27)